In [1]:
import dask.dataframe as dd
import matplotlib.pyplot as plt
import os
from sklearn.metrics import r2_score
import dask_ml
import dask
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.preprocessing import QuantileTransformer, PowerTransformer

import warnings
warnings.filterwarnings("ignore", category=UserWarning, message=".*Sending large graph.*")

from dask.distributed import Client, LocalCluster
import dask.multiprocessing

cluster = LocalCluster(processes=True,n_workers=6, threads_per_worker=1)
client = Client(cluster)
import sys
import pickle 

from data_helpers import *
from metrics import *

In [2]:
cluster = LocalCluster(processes=True,n_workers=6, threads_per_worker=1)
client = Client(cluster)

/Users/matthiaskargl/anaconda3/envs/leap/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 52850 instead
  warnings.warn(


In [3]:
folders = [
    'train0_25',
    'train25_50',
    'train50_75',
    'train75_100'
]

# Read Parquet files from each folder into Dask DataFrames
dfs = [dd.read_parquet(folder) for folder in folders]

# Concatenate all DataFrames into a single DataFrame
data = dd.concat(dfs)

In [4]:
with open('meanDict_allT.pkl', 'rb') as f:
    meanDict = pickle.load(f)

with open('stdDict_allT.pkl', 'rb') as f:
    stdDict = pickle.load(f)

with open('minVal_allT2.pkl', 'rb') as f:
    minDict = pickle.load(f)

with open('zScore_allT2.pkl', 'rb') as f:
    zscoreDict = pickle.load(f)

In [4]:
f = 'ptend_q0002_26'
nStd = 3
outliers1 = data.loc[((data[f]) > (meanDict[f]+ nStd*stdDict[f])) | ((data[f]) < (meanDict[f]- nStd*stdDict[f]))].compute()

In [14]:
f = 'ptend_q0002_26'
outliers1['z_score'+'_'+f] = (outliers1[f] - meanDict[f]) / stdDict[f]
outliers1

,sample_id,state_t_0,state_t_1,state_t_2,state_t_3,state_t_4,state_t_5,state_t_6,state_t_7,state_t_8,...,cam_out_PRECSC,cam_out_PRECC,cam_out_SOLS,cam_out_SOLL,cam_out_SOLSD,cam_out_SOLLD,weight,norm_weight,weight_ptend_q0002_26,z_score_ptend_q0002_26
3322,train_2182048,209.166177,224.938035,228.018635,243.349261,249.205380,258.411021,258.174338,250.076507,238.553741,...,0.0,0.000000e+00,0.000000,0.000000,0.000000,0.000000,7.262075e+02,0.000028,1.319408e+01,-3.632365
3324,train_2182050,212.593345,224.570564,225.798274,241.962484,248.586238,259.734963,259.203696,250.713844,240.306812,...,0.0,8.441302e-07,0.000000,0.000000,0.000000,0.000000,2.940487e+04,0.001120,2.911401e+02,-17.062828
412,train_6324584,208.606895,217.759996,228.922824,240.747778,251.782972,259.650411,257.098981,248.337107,238.889787,...,0.0,2.201517e-08,12.577186,15.537631,156.966297,87.991427,3.832313e+03,0.000146,5.726821e+02,-23.930777
413,train_6324585,207.772082,222.184756,227.297412,244.104153,253.266909,261.790015,257.548767,248.192046,238.930674,...,0.0,7.918069e-09,0.299567,0.287554,117.186860,63.594151,2.091450e+05,0.007967,7.025328e+04,-265.053347
414,train_6324586,209.154646,214.345825,215.402194,235.082479,246.323429,257.803212,257.298494,250.705021,241.359388,...,0.0,1.499519e-07,0.836154,0.552409,122.223853,48.861380,1.974914e+05,0.007523,6.921636e+04,-263.090015
415,train_6324587,214.325912,216.812184,219.050512,239.387488,250.106953,260.093061,258.117969,249.916448,240.494259,...,0.0,8.501923e-07,2.777748,2.126367,99.493859,64.781742,2.625169e+07,1.000000,9.322382e+06,-3053.257596
416,train_6324588,209.418035,221.925841,228.186056,245.657224,254.025303,261.152455,256.448109,247.930315,239.485777,...,0.0,2.742143e-12,62.412518,31.248308,137.572936,77.750318,3.340441e+03,0.000127,1.009287e+03,-31.769281
418,train_6324590,215.894711,221.089272,226.593576,241.718021,253.644508,260.554618,257.447209,249.086108,239.758561,...,0.0,2.186469e-07,2.918474,0.148706,100.370888,47.422220,2.923155e+05,0.011135,1.080387e+05,-328.692423
428,train_6324600,209.130147,203.022336,217.371903,232.711997,247.772427,257.551078,258.733363,252.848029,243.130269,...,0.0,0.000000e+00,308.574484,253.842312,97.460271,65.742195,1.873470e+03,0.000071,2.287404e+02,-15.124167
429,train_6324601,211.203156,207.822161,219.717064,234.687272,249.687313,258.581089,258.854590,252.354748,242.916018,...,0.0,1.215167e-08,113.361002,105.925653,223.303014,152.285950,8.850374e+04,0.003371,3.455859e+04,-185.899406


In [11]:
f = 'ptend_q0002_55'
nStd = 5
n_outliers5 = data.loc[((data[f]) > (meanDict[f]+ nStd*stdDict[f])) | ((data[f]) < (meanDict[f]- nStd*stdDict[f]))].shape[0].compute()
nStd = 3
n_outliers3 = data.loc[((data[f]) > (meanDict[f]+ nStd*stdDict[f])) | ((data[f]) < (meanDict[f]- nStd*stdDict[f]))].shape[0].compute()
n_data = data.shape[0].compute()

In [12]:
n_outliers3, n_outliers5, n_data

(200913, 50886, 10091520)

In [5]:
f = 'ptend_q0002_55'
nStd = 5
outliers2 = data.loc[((data[f]) > (meanDict[f]+ nStd*stdDict[f])) | ((data[f]) < (meanDict[f]- nStd*stdDict[f]))].compute()


2024-07-07 19:43:49,590 - distributed.scheduler - ERROR - Task ('repartitiontofewer-11f6950f38f8fc43a33f1b3154546504', 0) marked as failed because 4 workers died while trying to run it


KilledWorker: Attempted to run task ('repartitiontofewer-11f6950f38f8fc43a33f1b3154546504', 0) on 4 different workers, but all those workers died while running it. The last worker that attempt to run the task was tcp://127.0.0.1:49922. Inspecting worker logs is often a good next step to diagnose what went wrong. For more information see https://distributed.dask.org/en/stable/killed.html.

# calc z score

In [16]:
folders = [
    'train0_25',
    'train25_50',
    'train50_75',
    'train75_100'
]

for folder in folders:
    data = dd.read_parquet(folder)

    for f in ['ptend_q0002_25']:
        data['zscore_'+f] = (data[f] - meanDict[f])/stdDict[f]
    
    data.to_parquet(folder)

In [18]:
len(allT2)

308

# check z dict

In [10]:
data['zscore_ptend_q0002_55'].max().compute(),data['zscore_ptend_q0002_55'].min().compute(),data['zscore_ptend_q0002_26'].max().compute(),data['zscore_ptend_q0002_26'].min().compute()

(38.92672935483815,
 -16.994502228119632,
 0.0005143997657241457,
 -3053.2575964604935)

In [22]:
data['zscore_ptend_q0002_25'].max().compute(),data['zscore_ptend_q0002_25'].min().compute()

(0.0005454983747547031, -2984.371459556509)

In [24]:
for f in ['ptend_q0002_24']:
    data['zscore_'+f] = (data[f] - meanDict[f])/stdDict[f]

data['zscore_'+f].min().compute(),data['zscore_'+f].max().compute()

(-2647.380789565388, 0.0009180437887468676)

In [11]:
f = 'ptend_q0002_26'
outliers = data.loc[abs(data['zscore_'+f]) > 5].compute()

# zscore dict

In [31]:
f = 'ptend_q0002_26'
data.loc[((data['zscore_'+f]) < 5) | ((data['zscore_'+f]) > 5)].shape[0].compute()

Task exception was never retrieved
future: <Task finished name='Task-3441365' coro=<Client._gather.<locals>.wait() done, defined at /Users/matthiaskargl/anaconda3/envs/leap/lib/python3.10/site-packages/distributed/client.py:2197> exception=AllExit()>
Traceback (most recent call last):
  File "/Users/matthiaskargl/anaconda3/envs/leap/lib/python3.10/site-packages/distributed/client.py", line 2206, in wait
    raise AllExit()
distributed.client.AllExit


10091520

In [5]:
#zscoreDict = dict()
for f in allT2:
    if f not in zscoreDict.keys():
        data['zscore_'+f] = (data[f] - meanDict[f])/stdDict[f]
        zscoreDict[f] = {'min':data['zscore_'+f].min().compute(), 'max':data['zscore_'+f].max().compute()}
        print(f, zscoreDict[f])
    else:
        print('skipped', f)

skipped ptend_t_0
skipped ptend_t_1
skipped ptend_t_2
skipped ptend_t_3
skipped ptend_t_4
skipped ptend_t_5
skipped ptend_t_6
skipped ptend_t_7
skipped ptend_t_8
skipped ptend_t_9
skipped ptend_t_10
skipped ptend_t_11
skipped ptend_t_12
skipped ptend_t_13
skipped ptend_t_14
skipped ptend_t_15
skipped ptend_t_16
skipped ptend_t_17
skipped ptend_t_18
skipped ptend_t_19
skipped ptend_t_20
skipped ptend_t_21
skipped ptend_t_22
skipped ptend_t_23
skipped ptend_t_24
skipped ptend_t_25
skipped ptend_t_26
skipped ptend_t_27
skipped ptend_t_28
skipped ptend_t_29
skipped ptend_t_30
skipped ptend_t_31
skipped ptend_t_32
skipped ptend_t_33
skipped ptend_t_34
skipped ptend_t_35
skipped ptend_t_36
skipped ptend_t_37
skipped ptend_t_38
skipped ptend_t_39
skipped ptend_t_40
skipped ptend_t_41
skipped ptend_t_42
skipped ptend_t_43
skipped ptend_t_44
skipped ptend_t_45
skipped ptend_t_46
skipped ptend_t_47
skipped ptend_t_48
skipped ptend_t_49
skipped ptend_t_50
skipped ptend_t_51
skipped ptend_t_52
ski

In [7]:
zscoreDict

{'ptend_t_0': {'min': -63.21447975879488, 'max': 245.67247936945668},
 'ptend_t_1': {'min': -85.03918460626298, 'max': 485.3715638134578},
 'ptend_t_2': {'min': -126.60629741789786, 'max': 68.02051460930264},
 'ptend_t_3': {'min': -63.63672121888045, 'max': 2.8775431157252003},
 'ptend_t_4': {'min': -3.6853286303219415, 'max': 2.1461468624645654},
 'ptend_t_5': {'min': -2.5782653965224633, 'max': 2.294037629450322},
 'ptend_t_6': {'min': -2.0357513955629463, 'max': 2.657069630332726},
 'ptend_t_7': {'min': -2.546213690396981, 'max': 2.971244695190985},
 'ptend_t_8': {'min': -2.530318347897266, 'max': 3.1563311665414484},
 'ptend_t_9': {'min': -2.450968150453563, 'max': 3.372380426837497},
 'ptend_t_10': {'min': -2.5501623751607663, 'max': 3.3834928657381136},
 'ptend_t_11': {'min': -2.5926513754892664, 'max': 3.3065237327024843},
 'ptend_t_12': {'min': -2.620270894761342, 'max': 3.3768677441385972},
 'ptend_t_13': {'min': -2.971609148196438, 'max': 3.2113149540255908},
 'ptend_t_14': {

In [8]:
with open('zScore_allT2.pkl', 'wb') as f:
    pickle.dump(zscoreDict, f)   

In [13]:
f = 'ptend_q0002_55'
outlier2 = data.loc[abs(data['zscore_'+f]) > 30].compute()

In [15]:
outlier2

,sample_id,state_t_0,state_t_1,state_t_2,state_t_3,state_t_4,state_t_5,state_t_6,state_t_7,state_t_8,...,cam_out_PRECC,cam_out_SOLS,cam_out_SOLL,cam_out_SOLSD,cam_out_SOLLD,weight,norm_weight,weight_ptend_q0002_26,zscore_ptend_q0002_26,zscore_ptend_q0002_55
1693,train_1028936,221.208601,227.448212,235.642243,246.708969,256.622691,262.184891,259.401352,249.917317,238.691930,...,9.280329e-14,0.032660,0.609739,9.674005,6.353375,1154.684429,0.000044,2.646071e-07,0.000514,32.747714
3446,train_2043371,211.801267,227.893717,228.449389,241.408343,252.462005,257.903203,258.126331,252.480963,241.578580,...,9.115430e-07,17.950495,14.106293,70.392860,51.913388,14688.210471,0.000560,2.612900e-07,0.000511,30.212341
2778,train_2647297,215.401560,226.805432,232.538220,246.183895,257.577686,265.552226,265.269257,256.738459,244.829878,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,1365.002804,0.000052,2.646071e-07,0.000514,35.704629
2215,train_5185258,220.799354,233.221042,237.019025,243.782057,244.017413,256.226140,256.024662,251.622954,241.242152,...,4.019635e-07,0.000000,0.000000,0.000000,0.000000,5940.661221,0.000226,2.645489e-07,0.000514,35.635704
1891,train_5224176,215.657022,226.263426,234.215362,244.096132,254.275537,260.089025,258.369837,249.969983,240.538367,...,0.000000e+00,0.000225,0.000016,20.566001,12.207765,1281.493342,0.000049,2.646071e-07,0.000514,34.627326
3297,train_5652119,213.103191,230.286899,237.485179,242.426453,249.536412,257.916167,258.022113,254.685504,240.553405,...,9.773098e-11,0.000000,0.000000,0.000000,0.000000,1326.074083,0.000051,2.646071e-07,0.000514,33.444650
148,train_7188169,214.823838,225.482848,234.115124,246.699029,255.706254,263.184696,263.554557,255.725994,244.592522,...,0.000000e+00,1.212999,3.278566,8.400267,6.239881,1121.801842,0.000043,2.646071e-07,0.000514,31.241410
2759,train_7372248,224.981483,231.416195,236.199494,249.760761,261.431521,264.368992,256.114167,243.166669,234.444486,...,0.000000e+00,0.000013,0.000103,9.004354,9.049553,1784.220747,0.000068,2.646071e-07,0.000514,38.926729
